In [28]:
!pip install -q ultralytics

# Download files

In [29]:
import gdown

#download model
gdown.download_folder(id='1QVWjX8Aa2VvIbXjDY4DyqcV_Uqw6myX7', output="model", quiet=False, use_cookies=False)

#download test ds
gdown.download_folder(id='1LZIhiV9l82W4fNpfaBHyGvh8XBX6SoU2', output="test_folder", quiet=False, use_cookies=False)

Retrieving folder contents


Processing file 1vA70lsUOr-TYG7337eQ7Z-X4J7WbLvVD kuih-scratch.pt
Processing file 1_10GTTSqYCDbFgEdzRyU3YUXgaGc8aqN labels.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1vA70lsUOr-TYG7337eQ7Z-X4J7WbLvVD
From (redirected): https://drive.google.com/uc?id=1vA70lsUOr-TYG7337eQ7Z-X4J7WbLvVD&confirm=t&uuid=52fed62c-23c1-4b29-a914-25076b923831
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC_submission\naic\model\kuih-scratch.pt
100%|██████████| 125M/125M [00:01<00:00, 67.9MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1_10GTTSqYCDbFgEdzRyU3YUXgaGc8aqN
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC_submission\naic\model\labels.txt
100%|██████████| 124/124 [00:00<?, ?B/s] 
Download completed
Retrieving folder contents


Retrieving folder 1qO-cQtRdp-hLu6LMQ68OEv0zA791puJO test_images
Processing file 1cxftTp4A1_13gWe4QeAlhK9AcGzIDBEH Kek Lapis.jpg
Processing file 1aqVkDJiANm0MqLeTJALC1RuKZgt0IlI4 Kuih Kaswi Pandan.jpg
Processing file 11K7Hc9nt3Qllw4lG4ajxineVHvXKmuY6 Kuih Ketayap.jpg
Processing file 1qTcPAX5eSPLkAkhlyGfgooDXHZPFoJOM Kuih Lapis.jpg
Processing file 17CPxIkfU_tecxiF1-G4nXTD0Vv_1biB- Kuih Seri Muka.png
Processing file 1MEhhjAoBK66wtj6Xs1orsrA2vR9qLjhR Kuih Talam.jpg
Processing file 1KmI2JRpz4DnCAmPbCWoK31BJBH4ax8i7 Kuih Ubi Kayu.jpg
Processing file 1QFZdtM4ok-vH0kPgA7gdZQzT1aSppWcF Onde-Onde.jpg
Processing file 12ufiwhl6whVp33F05RLiDRieRSN13XlP test_labels.txt


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1cxftTp4A1_13gWe4QeAlhK9AcGzIDBEH
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC_submission\naic\test_folder\test_images\Kek Lapis.jpg
100%|██████████| 1.27M/1.27M [00:00<00:00, 19.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1aqVkDJiANm0MqLeTJALC1RuKZgt0IlI4
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC_submission\naic\test_folder\test_images\Kuih Kaswi Pandan.jpg
100%|██████████| 418k/418k [00:00<00:00, 8.39MB/s]
Downloading...
From: https://drive.google.com/uc?id=11K7Hc9nt3Qllw4lG4ajxineVHvXKmuY6
To: c:\Users\ochon\OneDrive - student.newinti.edu.my\NAIC_submission\naic\test_folder\test_images\Kuih Ketayap.jpg
100%|██████████| 373k/373k [00:00<00:00, 7.48MB/s]
Downloading...
From: https://drive.google.com/uc?id=1qTcPAX5eSPLkAkhlyGfgooDXHZPFoJOM
To: c:\Users\ochon\OneDrive - student.newinti.edu

['test_folder\\test_images\\Kek Lapis.jpg',
 'test_folder\\test_images\\Kuih Kaswi Pandan.jpg',
 'test_folder\\test_images\\Kuih Ketayap.jpg',
 'test_folder\\test_images\\Kuih Lapis.jpg',
 'test_folder\\test_images\\Kuih Seri Muka.png',
 'test_folder\\test_images\\Kuih Talam.jpg',
 'test_folder\\test_images\\Kuih Ubi Kayu.jpg',
 'test_folder\\test_images\\Onde-Onde.jpg',
 'test_folder\\test_labels.txt']

# Load model

In [30]:
from ultralytics import YOLO

model =  YOLO("model/kuih-scratch.pt")

# Load labels

In [31]:
with open('model/labels.txt') as f:
    labels = {i: line.strip().split(' ', 1)[1] for i, line in enumerate(f)}

# Load test images

In [32]:
import os

test_images = [os.path.join(r, f) for r, d, fs in os.walk('test_folder/test_images')
                                  for f in fs if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Predict

In [33]:
from collections import Counter

num_classes = 8
predictions = []

for path in test_images:
    results = model(path, verbose=False)
    pred_classes = results[0].boxes.cls.tolist()  # List of detected class IDs

    if len(pred_classes) > 0:
        counts = Counter(pred_classes)
        probs = [counts.get(i, 0) for i in range(num_classes)]
        probs = [p / sum(probs) for p in probs]  # normalize to sum to 1
        predicted_class_idx = int(max(counts, key=counts.get))
    else:
        probs = [0.0] * num_classes
        predicted_class_idx = -1

    predictions.append({
        'image': os.path.basename(path),
        'predicted_class_index': predicted_class_idx,
        'predicted_label': labels.get(predicted_class_idx, 'Unknown'),
        'class_probabilities': probs
    })

# Save results to CSV

In [34]:
import pandas as pd

results_df = pd.DataFrame(predictions)
print(results_df)

results_df.to_csv("predictions.csv", index=False)

                   image  predicted_class_index    predicted_label  \
0          Kek Lapis.jpg                      0          Kek Lapis   
1  Kuih Kaswi Pandan.jpg                      1  Kuih Kaswi Pandan   
2       Kuih Ketayap.jpg                      2       Kuih Ketayap   
3         Kuih Lapis.jpg                      3         Kuih Lapis   
4     Kuih Seri Muka.png                      4     Kuih Seri Muka   
5         Kuih Talam.jpg                      5         Kuih Talam   
6      Kuih Ubi Kayu.jpg                      6      Kuih Ubi Kayu   
7          Onde-Onde.jpg                      7          Onde-Onde   

                                 class_probabilities  
0           [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  
1           [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]  
2           [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]  
3           [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]  
4       [0.0, 0.0, 0.125, 0.0, 0.875, 0.0, 0.0, 0.0]  
5           [0.0, 0.0, 0.0, 0.0, 0.0, 1

# Map the true labels of the images in the testing folder to model's class index

In [35]:
df_test = pd.read_csv('test_folder/test_labels.txt')

filename_to_class = dict(zip(df_test['Filename'].str.strip(), df_test['Class'].str.strip()))

label_to_class_index = {v.strip(): k for k, v in labels.items()}

true_class_index = [
    label_to_class_index[filename_to_class[os.path.basename(path).strip()]]
    for path in test_images
]

In [36]:
from sklearn.metrics import (
    classification_report, accuracy_score,
    roc_auc_score, precision_recall_fscore_support
)
from sklearn.preprocessing import label_binarize
import numpy as np

results_df['true_class_index'] = true_class_index  ## changes here in v2 - refer to true_class_index
y_true = results_df['true_class_index'].astype(int).values
y_pred = results_df['predicted_class_index'].astype(int).values
y_probs = np.array(results_df['class_probabilities'].tolist())

## Quick fix for ROC Curve as I only have 3 classes here (DO NOT NEED THIS IF YOU HAVE 8 CLASSES IN YOUR TEST SET)
FULL_NUM_CLASSES = 8  # total number of possible classes

# Pad probability vectors to length 8
def pad_probs(probs, target_len=FULL_NUM_CLASSES):
    padded = np.zeros(target_len)
    padded[:len(probs)] = probs  # assumes probs are in order (class 0, 1, 2, ...)
    return padded

# Apply padding
y_probs_padded = np.array([pad_probs(p, FULL_NUM_CLASSES) for p in results_df['class_probabilities']])

# Update your DataFrame or use directly in metrics
y_probs = y_probs_padded


In [37]:
# Number of classes
n_classes = FULL_NUM_CLASSES
class_names = list(range(FULL_NUM_CLASSES))

# Accuracy
acc = accuracy_score(y_true, y_pred)
print(f"\n✅ Accuracy: {acc:.4f}")

# Precision, Recall, F1 per class & macro
prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, labels=class_names, average=None)
macro_prec, macro_rec, macro_f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

print("\n📊 Per-class metrics:")
for i, cls in enumerate(class_names):
    print(f"Class {cls}: Precision={prec[i]:.4f}, Recall={rec[i]:.4f}, F1={f1[i]:.4f}")

print(f"\n📦 Macro Precision: {macro_prec:.4f}, Macro Recall: {macro_rec:.4f}, Macro F1: {macro_f1:.4f}")

# ROC AUC (requires binarized labels)
y_true_bin = label_binarize(y_true, classes=class_names)

# ROC AUC per class and macro
try:
    auc_per_class = roc_auc_score(y_true_bin, y_probs, average=None, multi_class='ovr')
    auc_macro = roc_auc_score(y_true_bin, y_probs, average='macro', multi_class='ovr')

    print("\n🎯 ROC AUC per class:")
    for i, cls in enumerate(class_names):
        print(f"Class {cls}: AUC = {auc_per_class[i]:.4f}")

    print(f"\n🌐 Macro ROC AUC: {auc_macro:.4f}")

except Exception as e:
    print(f"⚠️ ROC AUC could not be computed: {e}")



✅ Accuracy: 1.0000

📊 Per-class metrics:
Class 0: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 1: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 2: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 3: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 4: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 5: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 6: Precision=1.0000, Recall=1.0000, F1=1.0000
Class 7: Precision=1.0000, Recall=1.0000, F1=1.0000

📦 Macro Precision: 1.0000, Macro Recall: 1.0000, Macro F1: 1.0000

🎯 ROC AUC per class:
Class 0: AUC = 1.0000
Class 1: AUC = 1.0000
Class 2: AUC = 1.0000
Class 3: AUC = 1.0000
Class 4: AUC = 1.0000
Class 5: AUC = 1.0000
Class 6: AUC = 1.0000
Class 7: AUC = 1.0000

🌐 Macro ROC AUC: 1.0000
